In [ ]:
import pygame
import random
from typing import List, Tuple, Optional

# =========================
# Konfigurasi & Warna
# =========================
FPS = 60

# Warna (R, G, B)
BG = (22, 24, 28)
GRID_LINE = (70, 74, 82)
COIN_COLOR = (240, 210, 60)

HUMAN_COLOR = (80, 200, 255)   # biru muda
AI_COLOR = (255, 120, 120)     # merah muda

TEXT = (235, 235, 240)
SUBTEXT = (190, 195, 205)
TOPBAR_BG = (32, 36, 42)
PANEL_BG = (28, 30, 36)
BUTTON_BG = (50, 54, 62)
BUTTON_BG_HOVER = (70, 74, 86)

# Level setup: (name, grid_size, num_coins)
LEVELS = [
    ("Easy",   8,  15),
    ("Medium", 10, 15),
    ("Hard",   15, 15),
]

# UI metrics (untuk layout)
TOPBAR_H = 64
BOTTOM_H = 96
CELL_SIZE_DEFAULT = 48  # default; akan di-scale agar muat di layar

# Jeda kecil agar gerak AI terlihat
AI_DELAY_MS = 160

# =========================
# Tipe data
# =========================
Vec2 = Tuple[int, int]

# =========================
# Utilitas
# =========================
def clamp(n, lo, hi): return max(lo, min(hi, n))

def manhattan(a: Vec2, b: Vec2) -> int:
    return abs(a[0] - b[0]) + abs(a[1] - b[1])

def is_adjacent(a: Vec2, b: Vec2) -> bool:
    return manhattan(a, b) == 1

def nearest_coin(from_pos: Vec2, coins: List[Vec2]) -> Optional[Vec2]:
    if not coins:
        return None
    return min(coins, key=lambda c: manhattan(from_pos, c))

def step_towards(src: Vec2, dst: Vec2, grid_size: int) -> Vec2:
    x, y = src
    dx = dst[0] - x
    dy = dst[1] - y
    if abs(dx) >= abs(dy):
        x += 1 if dx > 0 else (-1 if dx < 0 else 0)
    else:
        y += 1 if dy > 0 else (-1 if dy < 0 else 0)
    x = clamp(x, 0, grid_size - 1)
    y = clamp(y, 0, grid_size - 1)
    return (x, y)

# =========================
# Game Round (satu level)
# =========================
class RoundGame:
    def __init__(self, grid_size: int, num_coins: int):
        self.grid_size = grid_size
        self.num_coins = num_coins
        self.reset()

    @property
    def human_start(self) -> Vec2:
        return (0, self.grid_size - 1)  # kiri-bawah

    @property
    def ai_start(self) -> Vec2:
        return (self.grid_size - 1, 0)  # kanan-atas

    def _spawn_coins(self) -> List[Vec2]:
        taken = {self.human, self.ai}
        coins = set()
        while len(coins) < self.num_coins:
            c = (random.randrange(self.grid_size), random.randrange(self.grid_size))
            if c not in taken and c not in coins:
                coins.add(c)
        return sorted(list(coins))

    def reset(self):
        self.human: Vec2 = self.human_start
        self.ai: Vec2 = self.ai_start
        self.coins: List[Vec2] = self._spawn_coins()
        self.human_score = 0
        self.ai_score = 0
        self.turn = "HUMAN"  # or "AI"
        self.over = False
        self.result = None  # "HUMAN_WIN" / "AI_WIN" / "DRAW"

    def _collect_if_any(self, who: str):
        pos = self.human if who == "HUMAN" else self.ai
        if pos in self.coins:
            self.coins.remove(pos)
            if who == "HUMAN":
                self.human_score += 1
            else:
                self.ai_score += 1

    def _check_over(self):
        if not self.coins:
            self.over = True
            if self.human_score > self.ai_score:
                self.result = "HUMAN_WIN"
            elif self.ai_score > self.human_score:
                self.result = "AI_WIN"
            else:
                self.result = "DRAW"

    # === Human & AI turn ===
    def human_try_move_dir(self, dx: int, dy: int):
        if self.over or self.turn != "HUMAN":
            return False
        # target cell
        tx = clamp(self.human[0] + dx, 0, self.grid_size - 1)
        ty = clamp(self.human[1] + dy, 0, self.grid_size - 1)
        target = (tx, ty)
        if is_adjacent(self.human, target):
            self.human = target
            self._collect_if_any("HUMAN")
            self._check_over()
            if not self.over:
                self.turn = "AI"
            return True
        return False

    def ai_step(self):
        if self.over or self.turn != "AI":
            return
        if not self.coins:
            self._check_over()
            return
        target = nearest_coin(self.ai, self.coins)
        if target is None:
            self._check_over()
            return
        self.ai = step_towards(self.ai, target, self.grid_size)
        self._collect_if_any("AI")
        self._check_over()
        if not self.over:
            self.turn = "HUMAN"

# =========================
# Game Manager (semua level)
# =========================
class CoinHuntGame:
    def __init__(self):
        self.level_idx = 0
        self.round: Optional[RoundGame] = None
        self.state = "MENU"  # MENU, PLAYING, ROUND_END, GAME_WIN, GAME_LOSE
        self.ai_timer = 0

        # UI compute overall window size dynamically to fit largest grid
        largest_grid = max(l[1] for l in LEVELS)  # size from tuples
        self.cell = self._pick_cell_size(largest_grid)
        self.board_px = largest_grid * self.cell
        self.width = self.board_px
        self.height = TOPBAR_H + self.board_px + BOTTOM_H

        # pygame screen & fonts
        self.screen = None
        self.font_title = None
        self.font = None
        self.font_small = None
        self.font_big = None

    def _pick_cell_size(self, grid):
        # coba agar muat di 1000 px lebar
        target_board_px = 960
        size = max(28, min(56, target_board_px // grid))
        return size

    def _current_level_conf(self):
        name, size, coins = LEVELS[self.level_idx]
        return name, size, coins

    def start_level(self):
        _, size, coins = self._current_level_conf()
        self.round = RoundGame(size, coins)
        self.state = "PLAYING"
        self.ai_timer = 0

    def reset_all(self):
        self.level_idx = 0
        self.round = None
        self.state = "MENU"
        self.ai_timer = 0

    def handle_human_move(self, key):
        if self.state != "PLAYING" or self.round is None:
            return

        moved = False
        if key == pygame.K_UP:
            moved = self.round.human_try_move_dir(0, -1)
        elif key == pygame.K_DOWN:
            moved = self.round.human_try_move_dir(0, 1)
        elif key == pygame.K_LEFT:
            moved = self.round.human_try_move_dir(-1, 0)
        elif key == pygame.K_RIGHT:
            moved = self.round.human_try_move_dir(1, 0)

        if moved and self.round.turn == "AI" and not self.round.over:
            self.ai_timer = AI_DELAY_MS

    def update_ai(self, dt_ms):
        if self.state != "PLAYING" or self.round is None:
            return
        if self.round.turn == "AI" and not self.round.over:
            self.ai_timer -= dt_ms
            if self.ai_timer <= 0:
                self.round.ai_step()

        # round selesai?
        if self.round.over and self.state == "PLAYING":
            # Aturan: Human harus MENANG untuk lanjut
            if self.round.result == "HUMAN_WIN":
                if self.level_idx + 1 < len(LEVELS):
                    self.state = "ROUND_END"  # layar interstitial Next Round
                else:
                    self.state = "GAME_WIN"   # menang semua level
            else:
                self.state = "GAME_LOSE"     # kalah atau draw ⇒ ulang dari Easy

    # =========================
    # Rendering helpers
    # =========================
    def grid_origin(self, grid_size):
        # area board tepat di bawah topbar
        ox = 0
        oy = TOPBAR_H
        return ox, oy

    def cell_rect(self, x, y, grid_size):
        ox, oy = self.grid_origin(grid_size)
        return pygame.Rect(ox + x * self.cell, oy + y * self.cell, self.cell, self.cell)

    def draw_topbar(self):
        pygame.draw.rect(self.screen, TOPBAR_BG, pygame.Rect(0, 0, self.width, TOPBAR_H))

        if self.state in ("PLAYING", "ROUND_END", "GAME_WIN", "GAME_LOSE"):
            level_name, size, _ = self._current_level_conf()
            txt = self.font.render(f"Round – {level_name}", True, TEXT)
            self.screen.blit(txt, (16, TOPBAR_H // 2 - txt.get_height() // 2))

        # Legend Human/AI di kanan
        legend_y = TOPBAR_H // 2
        # Human legend (kotak biru + label)
        hx = self.width - 280
        hy = legend_y - 10
        pygame.draw.rect(self.screen, HUMAN_COLOR, pygame.Rect(hx, hy, 20, 20), border_radius=4)
        lbl_h = self.font_small.render("Human", True, TEXT)
        self.screen.blit(lbl_h, (hx + 28, legend_y - lbl_h.get_height() // 2))

        # AI legend (segitiga merah + label)
        ax = self.width - 160
        ay = legend_y
        p1 = (ax, ay - 12)
        p2 = (ax - 12, ay + 12)
        p3 = (ax + 12, ay + 12)
        pygame.draw.polygon(self.screen, AI_COLOR, [p1, p2, p3])
        lbl_ai = self.font_small.render("AI", True, TEXT)
        self.screen.blit(lbl_ai, (ax + 20, legend_y - lbl_ai.get_height() // 2))

    def draw_board(self):
        level_name, grid_size, _ = self._current_level_conf()

        # Grid lines
        for x in range(grid_size + 1):
            px = x * self.cell
            pygame.draw.line(self.screen, GRID_LINE,
                             (px, TOPBAR_H), (px, TOPBAR_H + grid_size * self.cell), 1)
        for y in range(grid_size + 1):
            py = TOPBAR_H + y * self.cell
            pygame.draw.line(self.screen, GRID_LINE, (0, py), (grid_size * self.cell, py), 1)

        # Coins
        for c in self.round.coins:
            r = self.cell // 4
            rect = self.cell_rect(*c, grid_size)
            cx = rect.centerx
            cy = rect.centery
            pygame.draw.circle(self.screen, COIN_COLOR, (cx, cy), r)

        # Human (kotak rounded)
        hx, hy = self.round.human
        hrect = self.cell_rect(hx, hy, grid_size)
        pad = max(4, self.cell // 8)
        hrect.inflate_ip(-2 * pad, -2 * pad)
        pygame.draw.rect(self.screen, HUMAN_COLOR, hrect, border_radius=8)

        # AI (segitiga)
        ax, ay = self.round.ai
        arect = self.cell_rect(ax, ay, grid_size)
        p1 = (arect.centerx, arect.top + pad)
        p2 = (arect.left + pad, arect.bottom - pad)
        p3 = (arect.right - pad, arect.bottom - pad)
        pygame.draw.polygon(self.screen, AI_COLOR, [p1, p2, p3])

    def draw_bottom_panel(self):
        pygame.draw.rect(self.screen, PANEL_BG,
                         pygame.Rect(0, TOPBAR_H + self.board_px, self.width, BOTTOM_H))

        # Score kiri: Human
        txt_h = self.font.render(f"Human: {self.round.human_score}", True, TEXT)
        self.screen.blit(txt_h, (16, TOPBAR_H + self.board_px + 12))

        # Score kanan: AI
        txt_a = self.font.render(f"AI: {self.round.ai_score}", True, TEXT)
        self.screen.blit(txt_a, (self.width - txt_a.get_width() - 16, TOPBAR_H + self.board_px + 12))

        # Turn besar di tengah, warna mengikuti pemilik turn
        if not self.round.over:
            who = self.round.turn
            color = HUMAN_COLOR if who == "HUMAN" else AI_COLOR
            label = "Your Turn" if who == "HUMAN" else "AI Turn"
            t = self.font_big.render(label, True, color)
            self.screen.blit(t, (self.width // 2 - t.get_width() // 2, TOPBAR_H + self.board_px + 8))
        else:
            # Round result
            label = "You Win!" if self.round.result == "HUMAN_WIN" else ("You Lose" if self.round.result == "AI_WIN" else "Draw")
            color = HUMAN_COLOR if self.round.result == "HUMAN_WIN" else (AI_COLOR if self.round.result == "AI_WIN" else SUBTEXT)
            t = self.font_big.render(label, True, color)
            self.screen.blit(t, (self.width // 2 - t.get_width() // 2, TOPBAR_H + self.board_px + 8))

        tips = self.font_small.render("Use Arrow Keys to move  •  Press R to Restart", True, SUBTEXT)
        self.screen.blit(tips, (self.width // 2 - tips.get_width() // 2, TOPBAR_H + self.board_px + 56))

    def draw_button(self, rect: pygame.Rect, text: str, hover: bool = False):
        pygame.draw.rect(self.screen, BUTTON_BG_HOVER if hover else BUTTON_BG, rect, border_radius=10)
        lbl = self.font.render(text, True, TEXT)
        self.screen.blit(lbl, (rect.centerx - lbl.get_width() // 2, rect.centery - lbl.get_height() // 2))

    def draw_menu(self):
        # background
        self.screen.fill(BG)

        # title
        title = self.font_title.render("COIN HUNT", True, TEXT)
        self.screen.blit(title, (self.width // 2 - title.get_width() // 2, self.height // 3 - 40))

        subtitle = self.font.render("Get Ready to Start", True, SUBTEXT)
        self.screen.blit(subtitle, (self.width // 2 - subtitle.get_width() // 2, self.height // 3 + 24))

        # start button
        btn_w, btn_h = 260, 56
        btn_rect = pygame.Rect(self.width // 2 - btn_w // 2, self.height // 2, btn_w, btn_h)
        mx, my = pygame.mouse.get_pos()
        hover = btn_rect.collidepoint(mx, my)
        self.draw_button(btn_rect, "Click to Begin", hover)

        return btn_rect

    def draw_interstitial_next(self):
        # layar setelah menang round (bukan hard)
        self.screen.fill(BG)
        t = self.font_big.render("Round Clear!", True, HUMAN_COLOR)
        self.screen.blit(t, (self.width // 2 - t.get_width() // 2, self.height // 3 - 24))

        nxt = self.font.render("Click to continue to next round", True, TEXT)
        self.screen.blit(nxt, (self.width // 2 - nxt.get_width() // 2, self.height // 3 + 24))

        btn_w, btn_h = 320, 56
        btn_rect = pygame.Rect(self.width // 2 - btn_w // 2, self.height // 2, btn_w, btn_h)
        mx, my = pygame.mouse.get_pos()
        hover = btn_rect.collidepoint(mx, my)
        self.draw_button(btn_rect, "Next Round", hover)
        return btn_rect

    def draw_end_screen(self, win: bool):
        self.screen.fill(BG)
        title = "You beat all rounds!" if win else "You lost the challenge"
        color = HUMAN_COLOR if win else AI_COLOR
        t = self.font_big.render(title, True, color)
        self.screen.blit(t, (self.width // 2 - t.get_width() // 2, self.height // 3 - 24))

        sub = self.font.render("Play Again to start from Easy", True, TEXT)
        self.screen.blit(sub, (self.width // 2 - sub.get_width() // 2, self.height // 3 + 24))

        btn_w, btn_h = 220, 56
        btn_rect = pygame.Rect(self.width // 2 - btn_w // 2, self.height // 2, btn_w, btn_h)
        mx, my = pygame.mouse.get_pos()
        hover = btn_rect.collidepoint(mx, my)
        self.draw_button(btn_rect, "Play Again", hover)
        return btn_rect

    # =========================
    # Main rendering dispatcher
    # =========================
    def render(self):
        if self.state == "MENU":
            btn = self.draw_menu()
            return ("MENU_BTN", btn)

        elif self.state == "PLAYING":
            self.screen.fill(BG)
            self.draw_topbar()
            self.draw_board()
            self.draw_bottom_panel()
            return (None, None)

        elif self.state == "ROUND_END":
            btn = self.draw_interstitial_next()
            return ("NEXT_BTN", btn)

        elif self.state == "GAME_WIN":
            btn = self.draw_end_screen(True)
            return ("AGAIN_BTN", btn)

        elif self.state == "GAME_LOSE":
            btn = self.draw_end_screen(False)
            return ("AGAIN_BTN", btn)

        return (None, None)

# =========================
# Main
# =========================
def main():
    pygame.init()
    random.seed()

    game = CoinHuntGame()
    game.screen = pygame.display.set_mode((game.width, game.height))
    pygame.display.set_caption("COIN HUNT — Human vs Greedy AI")

    # Fonts
    game.font_title = pygame.font.SysFont("bahnschrift", 48)
    game.font_big = pygame.font.SysFont("bahnschrift", 32)
    game.font = pygame.font.SysFont("bahnschrift", 22)
    game.font_small = pygame.font.SysFont("bahnschrift", 18)

    clock = pygame.time.Clock()
    running = True

    while running:
        dt = clock.tick(FPS)
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False

            # Mouse interactions (buttons)
            elif event.type == pygame.MOUSEBUTTONDOWN and event.button == 1:
                tag, rect = game.render()  # get current button rect after draw
                if rect and rect.collidepoint(event.pos):
                    if tag == "MENU_BTN":
                        # start from Easy
                        game.level_idx = 0
                        game.start_level()
                    elif tag == "NEXT_BTN":
                        # advance level
                        game.level_idx += 1
                        game.start_level()
                    elif tag == "AGAIN_BTN":
                        # restart from Easy
                        game.reset_all()

            elif event.type == pygame.KEYDOWN:
                if event.key == pygame.K_ESCAPE:
                    running = False
                elif event.key == pygame.K_r:
                    # restart current round (tetap di level yang sama)
                    if game.state == "PLAYING" and game.round:
                        name, size, coins = game._current_level_conf()
                        game.round = RoundGame(size, coins)
                    else:
                        # dari menu / end screen, R juga bisa mulai dari Easy
                        game.reset_all()

                # Arrow moves (saat PLAYING)
                elif event.key in (pygame.K_UP, pygame.K_DOWN, pygame.K_LEFT, pygame.K_RIGHT):
                    if game.state == "PLAYING":
                        game.handle_human_move(event.key)

        # Update AI
        game.update_ai(dt)

        # DRAW
        tag, _ = game.render()
        pygame.display.flip()

    pygame.quit()

if __name__ == "__main__":
    main()
